In [1]:
PROJECT_ID = 'dwh-siloam'
REGION = 'asia-southeast1'
print(f"Project ID: {PROJECT_ID}\nRegion: {REGION}")

# Initialize Vertex AI
from pathlib import Path
import vertexai
from google.cloud import aiplatform

print(f"Checking Credentials...")
if not any((Path.cwd()/"service_account").glob('*.json')):
    print("Service account folder is empty. Fallback using default gcloud account")
    aiplatform.init(project=PROJECT_ID, location=REGION)
    vertexai.init(project=PROJECT_ID, location=REGION)
else:
    print('Using service account credentials from service_account folder')
    from google.oauth2 import service_account
    sa_file = list((Path.cwd()/"service_account").glob('*.json'))[0]
    print(f"Using service account file: {sa_file}")
    credentials = service_account.Credentials.from_service_account_file(sa_file)
    aiplatform.init(project=PROJECT_ID, location=REGION, credentials=credentials)
    vertexai.init(project=PROJECT_ID, location=REGION, credentials=credentials)

# Import libraries
from langchain_google_vertexai import VertexAI, ChatVertexAI, create_structured_runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional
import requests

Project ID: dwh-siloam
Region: asia-southeast1
Checking Credentials...
Using service account credentials from service_account folder
Using service account file: c:\Others\chatbot-llm\service_account\dwh-siloam-99402e61edd2.json


In [75]:
from settings import CopilotSettings
import time

config = CopilotSettings()

In [28]:
llm = VertexAI(
    model_name=config.GCP_AGENT_MODEL_NAME, 
    temperature=0, 
    max_output_tokens=8192
)
# chat_llm = ChatVertexAI(
#     model_name=config.GCP_AGENT_MODEL_NAME, 
#     convert_system_message_to_human=True, 
#     temperature=0, 
#     max_output_tokens=8192
# )

In [6]:
prompt = ChatPromptTemplate.from_template(
"""
You are a helpful assistant.
Answer the question below using Indonesian language.
Do not make any assumptions. If you don't know the answer, just answer that you don't know.

{question}
"""
)

In [7]:
chain = (
    prompt
    | chat_llm
)

response = chain.invoke({"question": "Apa makanan kesukaanku?"})
print(response.content)

Maaf, saya tidak dapat menjawab pertanyaan Anda karena saya tidak memiliki informasi tentang makanan kesukaan Anda. Saya hanya dapat membantu Anda dengan pertanyaan yang berkaitan dengan informasi umum atau tugas yang dapat saya selesaikan dengan kemampuan saya saat ini. 

Apakah ada pertanyaan lain yang dapat saya bantu?


## Make a conversation scheme

In [8]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder

In [9]:
system_message = """
You are a helpful assistant.
Answer the question below using Indonesian language.
Do not make any assumptions. If you don't know the answer, just answer that you don't know.
"""

In [49]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_message
        ),
        MessagesPlaceholder(variable_name="message"),
    ]
)

chain = (
    prompt
    | llm
)

In [50]:
messages = [
    AIMessage(
        content="Halo, apa kabar?",
    ),
    HumanMessage(
        content="Baik."
    ),
    AIMessage(
        content="Ada yang bisa saya bantu?",
    ),
]

query = HumanMessage(
    content="Apa makanan kesukaanku?"
)

In [51]:
messages.append(query)
messages

[AIMessage(content='Halo, apa kabar?'),
 HumanMessage(content='Baik.'),
 AIMessage(content='Ada yang bisa saya bantu?'),
 HumanMessage(content='Apa makanan kesukaanku?')]

In [45]:
response = chain.invoke({"message": messages})

In [46]:
answer = response.split("AI:")[-1].strip()
print(answer)

Maaf, saya tidak bisa menjawab pertanyaan itu karena saya tidak memiliki informasi tentang makanan kesukaan Anda.


## Start FU conversation here
Every you asked different question, edit `fu_query` variable. And run the cell below again.

In [81]:
fu_query = "Apakah perlu micin?"

start = time.time()
answer = AIMessage(
    content=answer
)
query = HumanMessage(
    content=fu_query
)
messages.append(answer)
messages.append(query)

response = chain.invoke({"message": messages})
if "AI:" in response:
    answer = response.split("AI:")[-1].strip()
else:
    answer = response
print(response)
print("Elapsed time:", round(time.time() - start, 3), "[sec]")

 AI: Penggunaan micin atau monosodium glutamat (MSG) dalam masakan soto ayam tergantung pada selera masing-masing orang. Micin merupakan penyedap rasa yang dapat membuat masakan menjadi lebih gurih dan lezat. Namun, penggunaan micin yang berlebihan dapat berdampak buruk bagi kesehatan. Oleh karena itu, sebaiknya gunakan micin dalam jumlah yang wajar dan tidak berlebihan.
Elapsed time: 1.865 [sec]


Check the question history here.

In [83]:
len(messages)

52